In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

C:\Users\tetex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = '../data/Sample_Books_rating.csv'
data = pd.read_csv(file_path)

data = data.head(3000)

In [3]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=5)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Data processing
X = list(data["revue/texte"])
y = np.array([int(i) - 1 for i in data["revue/score"]])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0)

In [5]:
def tokenize(inputs):
    return tokenizer(inputs, padding=True, truncation=True, max_length=512, return_tensors="tf")

X_train_tokenized = tokenize(X_train)
X_val_tokenized = tokenize(X_val)

In [6]:
# Convert BatchEncoding to a hashable type
X_train_tokenized_hashable = (X_train_tokenized['input_ids'], X_train_tokenized['token_type_ids'], X_train_tokenized['attention_mask'])
X_val_tokenized_hashable = (X_val_tokenized['input_ids'], X_val_tokenized['token_type_ids'], X_val_tokenized['attention_mask'])

# Convert labels to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tokenized_hashable, y_train)).shuffle(len(X_train)).batch(8)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_tokenized_hashable, y_val)).batch(8)

In [7]:
# Model compilation
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
# Model training
model.fit(train_dataset, epochs=3, validation_data=val_dataset, batch_size=32)

Epoch 1/3




263/263 [==============================] - 2765s 10s/step - loss: 1.2039 - accuracy: 0.5857 - val_loss: 1.1874 - val_accuracy: 0.6000
Epoch 2/3
263/263 [==============================] - 2746s 10s/step - loss: 1.1968 - accuracy: 0.5919 - val_loss: 1.1924 - val_accuracy: 0.6000
Epoch 3/3
263/263 [==============================] - 2740s 10s/step - loss: 1.1910 - accuracy: 0.5910 - val_loss: 1.1728 - val_accuracy: 0.6000


In [9]:
# Evaluate the model
eval_loss, eval_accuracy = model.evaluate(val_dataset)
print(f"Eval Loss: {eval_loss}, Eval Accuracy: {eval_accuracy}")

113/113 [==============================] - 315s 3s/step - loss: 1.1728 - accuracy: 0.6000
Eval Loss: 1.1728307008743286, Eval Accuracy: 0.6000000238418579
